## Steganography on JPEG file
*Ernest Warzocha, SWPS University 2024*

An experiment to encode a text message into image file. Steganography is a technique to avoid detection of encrypted data. It is much harder to detect if we don't know if there is any message at all.

### **Steps to encode the message:**
1. **Get bits of each letter** (char is exactly one byte in utf-8) and store them in a one dimensional array.
2. Prepend a message length at the beginning so we know how many bits to count while decoding.
3. **Image file is one dimensional R, G, B, R, G, B... pixel array. We chose to encode n * 3 value.** In this case it's red color of each pixel.
4. **If color value is odd and message bit equals 0 - do nothing. If color bit is even, make it odd. Do the opposite when bit value is equal to 1.**

**Decoding is reversed. Read the parity of pixel color sequence.** Please check a similar approach in bitmap file example. It includes AES encryption.

*Technically after uploading the code it can't be called steganography anymore. Algorithm is trivial but if I would make a more sophisticated encryption... I wouldn't upload it to public.*

*The image must be sent as it is in form of directly uploaded file. If you upload it into any platform, an additional compression might occur, eventually altering the pixel value. In some cases it might be considered as an additional security of message.*


In [30]:
MESSAGE = "This is a secret message"
IMG_DIR = "sample_image.jpg"

In [31]:
%load_ext autoreload
%autoreload 2

from PIL import Image
from bitarray import bitarray
import numpy as np
from utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Encode the message into binary format
*You could change the endianness for additional encryption step but it requires changes in decoding function.*

In [32]:
test = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1]


def encode_message(msg):
    encoded_msg = []

    msg_len_bits = number_to_bits_array(len(msg), 16)
    encoded_msg.extend(msg_len_bits)


    msg_bits = bitarray(endian='big')
    msg_bits.frombytes(msg.encode('utf-8'))

    encoded_msg.extend(list(msg_bits))

    return encoded_msg

encoded_message_bits = encode_message(MESSAGE)

assert(encoded_message_bits == test)

### Apply message bits onto pixel values

The message maximum length is length max = (width * height) / (16 + (message length * 8))

**Possible changes in encoding:**
- Different color we are encoding to.
- Not starting from the beginning of image.
- Encode into every fourth or tenth pixel.
- Change endianness.
- AES encryption.
- Single message bit encoded into many image bits. Message "1" is 111 of each color in pixel etc.
- Change the axis of applying sequence on image.
- Use file name to store some information.

and many more...


In [33]:
def apply_message_into_image(img, msg_bits):

    img = np.array(img)
    img_shape = img.shape
    img_flat = img.flatten()

    for i, bit in enumerate(msg_bits):
        val_index = i * 3
        value = img_flat[val_index]
        img_flat[val_index] = set_bit(value, bit) #todo: do not overflow value 255

    encoded_image = img_flat.reshape(img_shape)

    return Image.fromarray(encoded_image)
        


img = Image.open(IMG_DIR)

encoded_image = apply_message_into_image(img, encoded_message_bits)

### Message decoding

In [34]:
def decode_message_from_image(img):

    decoded_msg = []

    img = np.array(img)
    img_flat = img.flatten()

    

    return decoded_msg
    # msg = ""

    # img = np.array(img)
    # img_flat = img.flatten()

    # len_bits = []

    # for i in range(16):

    #     len_bits.append(is_even(img_flat[i * 3]))

    # length = 0

    # for idx, bit in enumerate(reversed(len_bits)):
    #     length |= bit << idx

    # x = 0

    # for char_index in range(length):

    #     char_val = 0


    #     for pixel_index in range(8):

    #         val_index = (16 * 3) + (x * 3)

    #         char_val |= is_even(img_flat[val_index]) << (7 - pixel_index)

    #         x += 1
        
    #     msg += chr(char_val)

    # return msg

decode_message_from_image(encoded_image)

# display(encoded_image)

'This is a secret message'